In [1]:
import os
import sys

import logging
import re
import pathlib

import datetime

import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt

from pyplink import PyPlink
import seaborn as sns

import statsmodels.api as sm

from basic_tools import *

"""

jupyter nbconvert 5_association.ipynb --to script
python 5_association.py albumin 2 0

for i in {00..101};do python 5_association.py $i 2 1;done


for i in {00..10};do python 5_association.py $i 2 0;done
for i in {11..20};do python 5_association.py $i 2 0;done
for i in {21..30};do python 5_association.py $i 2 0;done
for i in {31..40};do python 5_association.py $i 2 0;done
for i in {41..50};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {61..70};do python 5_association.py $i 2 0;done
for i in {71..80};do python 5_association.py $i 2 0;done
for i in {91..90};do python 5_association.py $i 2 0;done
for i in {91..101};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done

for i in {00..101};do python 5_association.py $i 1 0,2;done
for i in {00..101};do python 5_association.py $i 1 1;done

for i in {00..20};do python 5_association.py $i 1 1;done
for i in {21..40};do python 5_association.py $i 1 1;done
for i in {41..60};do python 5_association.py $i 1 1;done
for i in {61..80};do python 5_association.py $i 1 1;done
for i in {81..101};do python 5_association.py $i 1 1;done

"""

'\n\njupyter nbconvert 5_association.ipynb --to script\npython 5_association.py albumin 2 0\n\nfor i in {00..101};do python 5_association.py $i 2 1;done\n\n\nfor i in {00..10};do python 5_association.py $i 2 0;done\nfor i in {11..20};do python 5_association.py $i 2 0;done\nfor i in {21..30};do python 5_association.py $i 2 0;done\nfor i in {31..40};do python 5_association.py $i 2 0;done\nfor i in {41..50};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {61..70};do python 5_association.py $i 2 0;done\nfor i in {71..80};do python 5_association.py $i 2 0;done\nfor i in {91..90};do python 5_association.py $i 2 0;done\nfor i in {91..101};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\n\nfor i in {00..101};do python 5_association.py $i 1 0,2;done\nfor i in {00..101};do py

In [2]:
plink_KCHIP_HLA_AA_SNP_1000G=PyPlink(plink_KCHIP_HLA_AA_SNP_1000G_path)
plink_KCHIP_HLA_AA_SNP_1000G_fam=plink_KCHIP_HLA_AA_SNP_1000G.get_fam().astype({'fid':str,'iid':str}).rename(columns={'fid':'FID','iid':'IID'})
plink_KCHIP_HLA_AA_SNP_1000G_bim=plink_KCHIP_HLA_AA_SNP_1000G.get_bim()

In [3]:
phenotypes=pd.read_csv(pheno_all_file_path,sep='\t')
phenotypes=phenotypes.set_index('ID').loc[plink_KCHIP_HLA_AA_SNP_1000G_fam['IID']]

binary_continuous_traits=phenotypes.columns.difference(['age','sex','cohort','diabetes'])

In [4]:
len(binary_continuous_traits)

97

In [13]:
phenotype_name='platelet'
data_out_assoc_phenotype_path=data_out_assoc_path+phenotype_name+'/'

In [22]:
step_idx_sub=2

In [23]:
result_merge=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.merge.result.tsv'.format(step_idx_sub),sep='\t')

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
result_merge.sort_values("P")

Unnamed: 0 phenotype_type phenotype_name  samples(case/control)  step  \
12142         700     continuous       platelet                  86087     2   
65635      648220     continuous       platelet                  86087     2   
65653      648400     continuous       platelet                  86087     2   
65652      648390     continuous       platelet                  86087     2   
65658      648450     continuous       platelet                  86087     2   
65633      648200     continuous       platelet                  86087     2   
12173         728     continuous       platelet                  86087     2   
52491         775     continuous       platelet                  86087     2   
24322      241650     continuous       platelet                  86087     2   
24282      241250     continuous       platelet                  86087     2   
24332      241750     continuous       platelet                  86087     2   
24321      241640     continuous       platelet                  86087     2   
24324      241670     continuous       platelet                  86087     2   
24290      241330     continuous       platelet                  86087     2   
24245      240880     continuous       platelet                  86087     2   
24256      240990     continuous       platelet                  86087     2   
24192      240350     continuous       platelet                  86087     2   
24208      240510     continuous       platelet                  86087     2   
24183      240260     continuous       platelet                  86087     2   
24188      240310     continuous       platelet                  86087     2   
24257      241000     continuous       platelet                  86087     2   
24203      240460     continuous       platelet                  86087     2   
24353      241960     continuous       platelet                  86087     2   
24296      241390     continuous       platelet                  86087     2   
24335      241780     continuous       platelet                  86087     2   
22276      221190     continuous       platelet                  86087     2   
22272      221150     continuous       platelet                  86087     2   
22282      221250     continuous       platelet                  86087     2   
24323      241660     continuous       platelet                  86087     2   
22275      221180     continuous       platelet                  86087     2   
...           ...            ...            ...                    ...   ...   
14757      146000     continuous       platelet                  86087     2   
37093      367300     continuous       platelet                  86087     2   
10905      109050     continuous       platelet                  86087     2   
20620      204630     continuous       platelet                  86087     2   
20685      205280     continuous       platelet                  86087     2   
40965      406020     continuous       platelet                  86087     2   
32724      323610     continuous       platelet                  86087     2   
18672      185150     continuous       platelet                  86087     2   
18088      179310     continuous       platelet                  86087     2   
647          6470     continuous       platelet                  86087     2   
639          6390     continuous       platelet                  86087     2   
33263      329000     continuous       platelet                  86087     2   
3219        32190     continuous       platelet                  86087     2   
32792      324290     continuous       platelet                  86087     2   
33237      328740     continuous       platelet                  86087     2   
33236      328730     continuous       platelet                  86087     2   
64957      641440     continuous       platelet                  86087     2   
33072      327090     continuous       platelet                  86087     2   
7056   

In [ ]:
GAT_result[((~GAT_result['marker_name'].str.contains('HLA',regex=False))|(GAT_result['marker_name'].str.contains('*',regex=False)))]

In [31]:
#result_merge[((~result_merge['marker_name'].str.contains('HLA',regex=False))|(result_merge['marker_name'].str.contains('*',regex=False)))]

In [32]:
assoc_result_list=[]
for phenotype_name in binary_continuous_traits:
    data_out_assoc_phenotype_path=data_out_assoc_path+phenotype_name+'/'

    for step_idx_sub in range(1,100):
        #print(step_idx_sub)
        if os.path.exists(data_out_assoc_phenotype_path+'step_{:02d}.cond.stop'.format(step_idx_sub)):
            print('meets end',step_idx_sub)
            break
        result_merge=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.merge.result.tsv'.format(step_idx_sub),sep='\t')
        result_merge=result_merge[((~result_merge['marker_name'].str.contains('HLA',regex=False))|(result_merge['marker_name'].str.contains('*',regex=False)))]
        assoc_result_list.append(result_merge.sort_values('P').iloc[0])

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


meets end 2


/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


meets end 3


/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


...

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivi

meets end 6


/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivi

meets end 6


/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
#assoc_result_concat

In [33]:
assoc_result_concat=pd.concat(assoc_result_list,axis=1).transpose()

In [34]:
assoc_result_concat.to_csv('data/result_concat_1_top_all.tsv',sep='\t')#,index=None)

In [12]:
assoc_result_concat#.head()

Unnamed: 0 phenotype_type                         phenotype_name  \
20816     144613     continuous                 FEV_over_FVC_predicted   
68685     475104     continuous                          FEV_predicted   
21318     169288     continuous                          FEV_predicted   
68741     475496     continuous                          FVC_predicted   
21307     169200     continuous                          FVC_predicted   
29549     204407     continuous                          MVV_predicted   
56431     445472         binary                    acute_liver_disease   
12173        728     continuous                                albumin   
68761     679480     continuous                                albumin   
22997     251240     continuous                                albumin   
17705     210576     continuous                                albumin   
54209     481860         binary                       allergic_disease   
12809     290996         binary                       allergic_disease   
30669     272754     continuous                                    alt   
68693     678800     continuous                                    alt   
51373        419     continuous                                    alt   
29461        198         binary  angina_pectoris_myocardial_infarction   
36489     325134         binary                              arthritis   
41723        791     continuous                                    ast   
63263    1498800     continuous                                    ast   
68693    2307920     continuous                                    ast   
38500     343233         binary                                 asthma   
671         4697         binary           benign_prostatic_hyperplasia   
37231     258076         binary                               big_baby   
64351     508304         binary                         bladder_cancer   
11573      92584         binary                         blood_in_urine   
38146     340047         binary                         blood_in_urine   
51369        417     continuous                                    bmi   
4332       95304     continuous                                    bmi   
68875    1565426     continuous                                    bmi   
...          ...            ...                                    ...   
41739        794     continuous                systolic_blood_pressure   
68714     611109         binary                            t2_diabetes   
35422     350590         binary                            t2_diabetes   
34182     304371         binary                         thyroid_cancer   
41723        791         binary                        thyroid_disease   
12142        700         binary                        thyroid_disease   
41723        791         binary                        thyroid_disease   
41723        791         binary                        thyroid_disease   
41723        791         binary                        thyroid_disease   
39995     356688     continuous                        total_bilirubin   
52491        775     continuous                      total_cholesterol   
61399    1394996     continuous                      total_cholesterol   
29520        167         binary             transient_ischemic_attacks   
52491        775     continuous                           triglyceride   
68714    1561723     continuous                           triglyceride   
34658     823080     continuous                           triglyceride   
69788     482825     continuous                                    tsh   
3634       25438         binary                           tuberculosis   
11836     106524         binary                urinary_tract_infection   
68715     611118     continuous                    waist_circumference   
12173        728     continuous                                    wbc   
42882    1019904     continuous                                    wbc   
32222    1

In [16]:
result_merge.sort_values('P').iloc[0]

Unnamed: 0                                                          308568
phenotype_type                                                  continuous
phenotype_name                                                      weight
samples(case/control)                                               124693
step                                                                     5
condition                6:28861320_A/G,6:30990019_A/G,6:32569297_A/G,6...
marker_name                                                 6:29905224_A/T
marker_name_pub                                                 rs28482016
note                                                     unphased bialleic
term                                                                   ADD
POS                                                               29905224
A1                                                                       T
A2                                                                       A
A1_freq_case             